In [1]:
import matplotlib.pyplot as plt 
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset=tf.keras.utils.get_file("aclImdb_v1.tar.gz", url, untar=True, cache_dir='.', cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset), 'aclImdb')

In [3]:
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [4]:
train_dir=os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [5]:
sample_file=os.path.join(train_dir, 'pos/1180_9.txt')
with open(sample_file) as f:
    print(f.read())

"Tulip" is on the "Australian All Shorts" video from "Tribe First Rites" showcasing the talents of first time directors.<br /><br />I wish more scripts had such excellent dialogue.<br /><br />I hope Rachel Griffiths has more stories to tell, she does it so well.


In [6]:
remove_dir=os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [7]:
batch_size=32
seed=42

raw_train_ds=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [8]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Review", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

In [9]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [10]:
raw_val_ds=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [11]:
raw_test_ds=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [12]:
def custom_standardization(input_data):
    lowercase=tf.strings.lower(input_data)
    stripped_html=tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [13]:
max_features=10000
sequence_lenght=250

vectorize_layer=TextVectorization(standardize=custom_standardization, max_tokens=max_features, output_mode='int', output_sequence_length=sequence_lenght)

In [14]:
train_text=raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [15]:
def vectorize_text(text, label):
    text=tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [16]:
text_batch, label_batch=next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />T

In [17]:
print("22 --->", vectorize_layer.get_vocabulary()[22])
print("222 -->", vectorize_layer.get_vocabulary()[222])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

22 ---> you
222 --> music
Vocabulary size: 10000


In [20]:
train_ds=raw_train_ds.map(vectorize_text)
val_ds=raw_val_ds.map(vectorize_text)
test_ds=raw_test_ds.map(vectorize_text)

In [21]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [22]:
embedding_dim=16

In [23]:
model=tf.keras.Sequential([
    layers.Embedding(max_features+1, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
             optimizer='adam',
             metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [25]:
epochs=10
history=model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
625/625 [==============================] - 980s 2s/step - loss: 0.6823 - binary_accuracy: 0.6127 - val_loss: 0.6164 - val_binary_accuracy: 0.7730
Epoch 2/10
625/625 [==============================] - 8s 13ms/step - loss: 0.5819 - binary_accuracy: 0.7815 - val_loss: 0.5004 - val_binary_accuracy: 0.8214
Epoch 3/10
625/625 [==============================] - 8s 13ms/step - loss: 0.4680 - binary_accuracy: 0.8340 - val_loss: 0.4216 - val_binary_accuracy: 0.8468
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.3920 - binary_accuracy: 0.8595 - val_loss: 0.3745 - val_binary_accuracy: 0.8596
Epoch 5/10
625/625 [==============================] - 7s 11ms/step - loss: 0.3444 - binary_accuracy: 0.8768 - val_loss: 0.3456 - val_binary_accuracy: 0.8670
Epoch 6/10
625/625 [==============================] - 5s 9ms/step - loss: 0.3120 - binary_accuracy: 0.8867 - val_loss: 0.3263 - val_binary_accuracy: 0.8716
Epoch 7/10
625/625 [==============================] - 7s 12m

In [27]:
loss, accuracy=model.evaluate(test_ds)
print('Loss: ', loss)
print('Accuracy: ', accuracy)

782/782 [==============================] - 858s 1s/step - loss: 0.3104 - binary_accuracy: 0.8735
Loss:  0.3103722929954529
Accuracy:  0.8734800219535828


In [28]:
history_dict=history.history
history_dict.keys()

dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

In [ ]:
acc=history_dict['binary_accuracy']
val_acc=history_dict['val_binary_accuracy']